# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("output_data/weather_data.csv")
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,punta arenas,-53.15,-70.92,39.20,80,20,10.29,CL,1587204927
1,gornyy,55.12,83.91,52.47,49,0,8.86,RU,1587204928
2,aguimes,27.91,-15.45,66.99,77,75,13.87,ES,1587204928
3,baker city,44.77,-117.83,28.40,74,1,1.83,US,1587204928
4,itarema,-2.92,-39.92,74.44,90,28,3.49,BR,1587204928


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]
rating = vacation_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Use loc with condition
criteria_df = vacation_df.loc[(vacation_df['Max Temp'] < 80)]
criteria_df = criteria_df.loc[(criteria_df['Max Temp'] > 70)]
criteria_df = criteria_df.loc[(criteria_df['Wind Speed'] < 10)]
criteria_df = criteria_df.loc[(criteria_df['Cloudiness'] == 0)]
# criteria_df = criteria_df.dropna
criteria_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,rikitea,-23.12,-134.97,77.04,68,0,3.91,PF,1587204929
21,athens,37.98,23.72,72.00,53,0,5.82,GR,1587204916
113,corinto,37.94,22.96,75.00,53,0,5.82,GR,1587204945
169,viransehir,37.24,39.76,73.27,34,0,7.87,TR,1587204954
213,richards bay,-28.78,32.04,77.70,62,0,6.22,ZA,1587204961
348,ararat,39.83,44.70,71.60,28,0,3.36,AM,1587204982
438,kroonstad,-27.65,27.23,71.01,69,0,3.00,ZA,1587204995
455,gat,31.61,34.76,75.99,28,0,1.99,IL,1587204997
466,bahia honda,22.90,-83.16,78.06,77,0,0.87,CU,1587204999
475,jalu,29.03,21.55,75.85,25,0,2.59,LY,1587205001


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = criteria_df

# New columng "Coordinates" to join Lat and Long together; easier when pulling from API
hotel_df["Coordinates"] = hotel_df["Lat"].map(str) + ", " + hotel_df["Lng"].map(str)

# List to hold the API search results
hotel_result = []

# Define the search parameters
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
search_radius = 5000
params = {"keyword" : target_type,
         "radius" : search_radius,
         "key" : g_key}

# Use a for loop to retrieve data and put into hotel_result
for index, row in hotel_df.iterrows():
    hotel_coord = row["Coordinates"]
    params["location"] = hotel_coord
    nearest_hotel = requests.get(base_url, params = params).json()
    hotel_result.append(nearest_hotel["results"][0]["name"])

In [7]:
# Add hotel result list to the DataFrame
hotel_df["Hotel Name"] = hotel_result
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Coordinates,Hotel Name
8,rikitea,-23.12,-134.97,77.04,68,0,3.91,PF,1587204929,"-23.12, -134.97",Pension Maro'i
21,athens,37.98,23.72,72.00,53,0,5.82,GR,1587204916,"37.98, 23.72",Herodion Hotel
113,corinto,37.94,22.96,75.00,53,0,5.82,GR,1587204945,"37.94, 22.96",Hotel King Saron Ξενοδοχείο
169,viransehir,37.24,39.76,73.27,34,0,7.87,TR,1587204954,"37.24, 39.76",Yukselhan Hotel
213,richards bay,-28.78,32.04,77.70,62,0,6.22,ZA,1587204961,"-28.78, 32.04",BON Hotel Waterfront Richards Bay
348,ararat,39.83,44.70,71.60,28,0,3.36,AM,1587204982,"39.83, 44.7",B&B Sunrise House Aygavan
438,kroonstad,-27.65,27.23,71.01,69,0,3.00,ZA,1587204995,"-27.65, 27.23",Arcadia Guesthouse Kroonstad
455,gat,31.61,34.76,75.99,28,0,1.99,IL,1587204997,"31.61, 34.76",Desert Gat
466,bahia honda,22.90,-83.16,78.06,77,0,0.87,CU,1587204999,"22.9, -83.16",Casa Gualberto - Deysi
475,jalu,29.03,21.55,75.85,25,0,2.59,LY,1587205001,"29.03, 21.55",Jalu Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
new_locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(new_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))